In [1]:
import sqlite3
import urllib.parse

In [83]:
conn = sqlite3.connect("outlets.db")
cursor = conn.cursor()

In [84]:
cursor.execute("SELECT address FROM outlets")

In [85]:
addresses = cursor.fetchall()

clean_addresses = []

for address in addresses:
    clean_address = str(address).strip("(),'")
    clean_addresses.append(urllib.parse.quote(clean_address))

In [74]:
cursor.execute("DELETE FROM outlets WHERE address IS NULL OR name IS NULL")

In [86]:
print(clean_addresses)

['No.41%2C%20Ground%20Floor%2C%20Jalan%20Komersial%2C%20Taman%20Kota%20Fesyen%20MITC%2C%2075450%2C%20Ayer%20Keroh%2C%20Melaka', 'JC%2023%2C%20Ground%20Floor%2C%20Jalan%20BMU%201%2C%20Bandar%20Baru%20Merlimau%20Utara%2C%2077300%2C%20Merlimau%2C%20Melaka', 'SI%2015%2C%20Ground%20Floor%2C%20Jalan%20Sungai%20Baru%20Ilir%20Jaya%20Utama%2C%20Taman%20Sungai%20Baru%20Ilir%20Jaya%2C%2078300%20Masjid%20Tanah%2C%20Melaka.', 'No%2039%20%28Ground%20Floor%29%2C%20Jalan%20Merdeka%201%2C%20Taman%20Merdeka%2C%20Hang%20Tuah%20Jaya%2C%2075350%20Batu%20Berendam%2C%20Melaka.', 'No%2017%20%28Ground%20Floor%29%2C%20Jalan%20Inang%201%2C%20Taman%20Paya%20Rumput%20Utama%2C%2076450%20Melaka.%20', 'G13%2C%20Ground%20Floor%2C%20ElementX%20%40%20Hatten%20City%2C%20Jalan%20Melaka%20Raya%2C%2023%2C%20Taman%20Melaka%20Raya%2C%2075000%20Melaka%20', 'No%2024%20%28Ground%20Floor%29%2C%20Jalan%20Kerambit%204%2C%20Bandar%20Baru%20Sungai%20Udang%2C%2076300%20Sungai%20Udang%2C%20Melaka.%20', 'B-001%28B%29%20Dataran%20Pahlawa

In [76]:
from dotenv import load_dotenv

load_dotenv()

True

In [77]:
import os

print(os.getenv("API_KEY"))

8dda525c4e9040b5bac4a997baf2aad5


In [78]:
import requests

geocode_api_url = "https://api.geoapify.com/v1/geocode/search?text={}&apiKey={}"

address_lon_lat = []

for address in clean_addresses:
    geocode_url = geocode_api_url.format(address, os.getenv("API_KEY"))
    geocode_request = requests.get(geocode_url)
    geocode_request_json = geocode_request.json()
    address_longitude = geocode_request_json["features"][0]["properties"]["lon"]
    address_latitude = geocode_request_json["features"][0]["properties"]["lat"]
    
    address_lon_lat.append([address_longitude, address_latitude])

In [79]:
print(address_lon_lat)

[[102.2915442, 2.255986], [102.425724, 2.147558], [102.1089793, 2.3522253], [102.2542092, 2.2507491], [102.2821695, 2.2723766], [-88.75381357309995, 44.38431855], [102.1338866, 2.2841096], [102.2515728, 2.1943035], [102.1883, 2.4364], [102.398694, 2.275298], [102.1089793, 2.3522253], [102.2710708, 2.2157944], [111.5402, -7.6279], [102.222656, 2.228419], [102.2375854, 2.1966038]]


In [47]:
cursor.execute("""
    ALTER TABLE outlets ADD lon REAL
""")

In [48]:
cursor.execute("""
    ALTER TABLE outlets ADD lat REAL
""")

In [87]:
for i, (lon,lat) in enumerate(address_lon_lat):
    cursor.execute("UPDATE outlets SET lon = ?, lat = ? WHERE id = ?", (lon, lat, i+1))

In [88]:
conn.commit()
conn.close()